In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

# Carrega a URL do banco de dados
load_dotenv('../.env')
DATABASE_URL = os.getenv("DATABASE_URL")

# Carrega os dados dos funcionários com as predições
engine = create_engine(DATABASE_URL)
df = pd.read_sql_query('SELECT * FROM employees', engine)
# Supondo que você já tenha rodado o script de predição em lote
df_preds = pd.read_sql_query('SELECT "EmployeeNumber", "predicted_probability" FROM predictions_batch', engine)
df_full = pd.merge(df, df_preds, on='EmployeeNumber', how='left')

# Separa os grupos
high_risk = df_full[df_full['predicted_probability'] > 0.90]
low_risk = df_full[df_full['predicted_probability'] < 0.10]

print(f"Total de funcionários de alto risco (>90%): {len(high_risk)}")
print(f"Total de funcionários de baixo risco (<10%): {len(low_risk)}")

# Compara a distribuição de uma feature suspeita, por exemplo, 'OverTime'
print("\n--- Comparação de Horas Extras (OverTime) ---")
print("Alto Risco:")
print(high_risk['OverTime'].value_counts(normalize=True))
print("\nBaixo Risco:")
print(low_risk['OverTime'].value_counts(normalize=True))

Total de funcionários de alto risco (>90%): 204
Total de funcionários de baixo risco (<10%): 1211

--- Comparação de Horas Extras (OverTime) ---
Alto Risco:
OverTime
Yes    0.602941
No     0.397059
Name: proportion, dtype: float64

Baixo Risco:
OverTime
No     0.76796
Yes    0.23204
Name: proportion, dtype: float64


In [2]:
import pandas as pd

# Carregue os seus dados brutos
#df = pd.read_csv('.../data/raw/WA_Fn-UseC_-HR-Employee-Attrition.csv')
df = pd.read_csv('../data/raw/WA_Fn-UseC_-HR-Employee-Attrition.csv')

# Verifique a contagem e a proporção da variável alvo
print("Contagem de Attrition:")
print(df['Attrition'].value_counts())
print("\nProporção de Attrition:")
print(df['Attrition'].value_counts(normalize=True))

Contagem de Attrition:
Attrition
No     1233
Yes     237
Name: count, dtype: int64

Proporção de Attrition:
Attrition
No     0.838776
Yes    0.161224
Name: proportion, dtype: float64


In [3]:
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
from pathlib import Path

# --- Configurações e Conexões ---

# Carrega a variável de ambiente do ficheiro .env na raiz do projeto
load_dotenv()
DATABASE_URL = os.getenv("DATABASE_URL")

# Caminho para o ficheiro CSV original
RAW_DATA_PATH = ('../data/raw/WA_Fn-UseC_-HR-Employee-Attrition.csv')

print("--- INICIANDO AUDITORIA FINAL DOS DADOS ---")

# --- Passo 1: Carregar Dados do PostgreSQL ---
print("\n1. Carregando dados do PostgreSQL...")
try:
    engine = create_engine(DATABASE_URL)
    with engine.connect() as conn:
        df_postgres = pd.read_sql_query('SELECT * FROM "employees"', conn)
    print(f"✅ Dados do PostgreSQL carregados com sucesso. Forma: {df_postgres.shape}")
except Exception as e:
    print(f"❌ Falha ao carregar dados do PostgreSQL: {e}")
    df_postgres = pd.DataFrame()


# --- Passo 2: Carregar Dados do Ficheiro CSV Original ---
print("\n2. Carregando dados do ficheiro CSV original...")
try:
    df_csv = pd.read_csv(RAW_DATA_PATH)
    print(f"✅ Dados do CSV carregados com sucesso. Forma: {df_csv.shape}")
except Exception as e:
    print(f"❌ Falha ao carregar dados do CSV: {e}")
    df_csv = pd.DataFrame()

# --- Passo 3: Comparação ---

if not df_postgres.empty and not df_csv.empty:
    print("\n--- 3. COMPARAÇÃO DOS DADOS ---")

    # a) Comparar o número de linhas e colunas
    if df_postgres.shape == df_csv.shape:
        print(f"✅ VERIFICAÇÃO DE DIMENSÃO: OK! Ambos têm {df_postgres.shape[0]} linhas e {df_postgres.shape[1]} colunas.")
    else:
        print(f"❌ ALERTA DE DIMENSÃO: PostgreSQL tem {df_postgres.shape}, enquanto o CSV tem {df_csv.shape}.")

    # b) Comparar as primeiras 5 linhas para inspeção visual
    print("\n--- Primeiras 5 linhas do PostgreSQL: ---")
    display(df_postgres.head())
    
    print("\n--- Primeiras 5 linhas do CSV Original: ---")
    display(df_csv.head())

    # c) Comparar estatísticas descritivas das colunas numéricas
    print("\n--- Estatísticas Descritivas do PostgreSQL: ---")
    display(df_postgres.describe().T)

    print("\n--- Estatísticas Descritivas do CSV Original: ---")
    display(df_csv.describe().T)

    # d) Verificação final de igualdade
    # Nota: isto pode dar 'False' se os tipos de dados foram ligeiramente alterados (ex: int64 vs int32),
    # mas se as verificações acima passarem, os dados estão consistentes.
    if df_postgres.equals(df_csv):
        print("\n✅ VERIFICAÇÃO DE CONTEÚDO: OK! Os DataFrames são idênticos.")
    else:
        print("\n⚠️ AVISO DE CONTEÚDO: Os DataFrames não são 100% idênticos. Verifique as estatísticas e os tipos de dados. Pequenas diferenças de tipo são normais.")
else:
    print("\nNão foi possível realizar a comparação pois um dos DataFrames está vazio.")

--- INICIANDO AUDITORIA FINAL DOS DADOS ---

1. Carregando dados do PostgreSQL...
✅ Dados do PostgreSQL carregados com sucesso. Forma: (1470, 35)

2. Carregando dados do ficheiro CSV original...
✅ Dados do CSV carregados com sucesso. Forma: (1470, 35)

--- 3. COMPARAÇÃO DOS DADOS ---
✅ VERIFICAÇÃO DE DIMENSÃO: OK! Ambos têm 1470 linhas e 35 colunas.

--- Primeiras 5 linhas do PostgreSQL: ---


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2



--- Primeiras 5 linhas do CSV Original: ---


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2



--- Estatísticas Descritivas do PostgreSQL: ---


,count,mean,std,min,25%,50%,75%,max
Age,1470.0,36.923810,9.135373,18.0,30.00,36.0,43.00,60.0
DailyRate,1470.0,802.485714,403.509100,102.0,465.00,802.0,1157.00,1499.0
DistanceFromHome,1470.0,9.192517,8.106864,1.0,2.00,7.0,14.00,29.0
Education,1470.0,2.912925,1.024165,1.0,2.00,3.0,4.00,5.0
EmployeeCount,1470.0,1.000000,0.000000,1.0,1.00,1.0,1.00,1.0
EmployeeNumber,1470.0,1024.865306,602.024335,1.0,491.25,1020.5,1555.75,2068.0
EnvironmentSatisfaction,1470.0,2.721769,1.093082,1.0,2.00,3.0,4.00,4.0
HourlyRate,1470.0,65.891156,20.329428,30.0,48.00,66.0,83.75,100.0
JobInvolvement,1470.0,2.729932,0.711561,1.0,2.00,3.0,3.00,4.0
JobLevel,1470.0,2.063946,1.106940,1.0,1.00,2.0,3.00,5.0



--- Estatísticas Descritivas do CSV Original: ---


,count,mean,std,min,25%,50%,75%,max
Age,1470.0,36.923810,9.135373,18.0,30.00,36.0,43.00,60.0
DailyRate,1470.0,802.485714,403.509100,102.0,465.00,802.0,1157.00,1499.0
DistanceFromHome,1470.0,9.192517,8.106864,1.0,2.00,7.0,14.00,29.0
Education,1470.0,2.912925,1.024165,1.0,2.00,3.0,4.00,5.0
EmployeeCount,1470.0,1.000000,0.000000,1.0,1.00,1.0,1.00,1.0
EmployeeNumber,1470.0,1024.865306,602.024335,1.0,491.25,1020.5,1555.75,2068.0
EnvironmentSatisfaction,1470.0,2.721769,1.093082,1.0,2.00,3.0,4.00,4.0
HourlyRate,1470.0,65.891156,20.329428,30.0,48.00,66.0,83.75,100.0
JobInvolvement,1470.0,2.729932,0.711561,1.0,2.00,3.0,3.00,4.0
JobLevel,1470.0,2.063946,1.106940,1.0,1.00,2.0,3.00,5.0



⚠️ AVISO DE CONTEÚDO: Os DataFrames não são 100% idênticos. Verifique as estatísticas e os tipos de dados. Pequenas diferenças de tipo são normais.


In [4]:
import joblib
import pandas as pd

# Carrega a lista de features que o modelo usou para ser treinado
try:
    feature_list = joblib.load('../artifacts/features/features.pkl')
    print("✅ Lista de features carregada com sucesso!")
    
    # Imprime a lista para inspeção
    print(f"\nO modelo foi treinado com as seguintes {len(feature_list)} features:")
    for feature in sorted(feature_list):
        print(f"- {feature}")

except FileNotFoundError:
    print("❌ ERRO: Ficheiro 'artifacts/features/features.pkl' não encontrado.")
    print("Por favor, execute o pipeline de treino primeiro com: poetry run python src/attrition/main.py run-pipeline")

✅ Lista de features carregada com sucesso!

O modelo foi treinado com as seguintes 48 features:
- Age
- BusinessTravel_Travel_Frequently
- BusinessTravel_Travel_Rarely
- DailyRate
- Department_Research & Development
- Department_Sales
- DistanceFromHome
- Education
- EducationField_Life Sciences
- EducationField_Marketing
- EducationField_Medical
- EducationField_Other
- EducationField_Technical Degree
- EmployeeNumber
- EnvironmentSatisfaction
- Gender
- HourlyRate
- JobInvolvement
- JobLevel
- JobRole_Human Resources
- JobRole_Laboratory Technician
- JobRole_Manager
- JobRole_Manufacturing Director
- JobRole_Research Director
- JobRole_Research Scientist
- JobRole_Sales Executive
- JobRole_Sales Representative
- JobSatisfaction
- MaritalStatus_Married
- MaritalStatus_Single
- MonthlyIncome
- MonthlyIncome_log
- MonthlyRate
- NumCompaniesWorked
- OverTime_Yes
- PercentSalaryHike
- PerformanceRating
- RelationshipSatisfaction
- StockOptionLevel
- TotalWorkingYears
- TotalWorkingYears_l